In [1]:
from lion_core.session.branch import Branch
from lion_service import iModel

In [2]:
gpt4o = iModel(
    provider="openai",
    api_key="OPENAI_API_KEY",
    task="chat",
    model='gpt-4o',
    interval_tokens=1_000_000,
    interval_requests=10_000,
    max_tokens=4_000,
    temperature=0.1,
)

In [3]:
from pydantic import BaseModel, Field
from lion_core.session.branch import Branch

class SelectionModel(BaseModel):    
    selected: list = Field([], description="select from [positive, negative, neutral]")

branch = Branch(
    operative_model=SelectionModel,
    imodel=gpt4o,
)

a = await branch.operate(
    instruction="Answer the philosophical question provided",
    guidance="Reply concisely and informatively",
    context=["What is the meaning of life?"],
    reason=True,
    temperature=0.3
)

In [4]:
from lionfuncs import as_readable_json
print(as_readable_json(a))

{
    "selected": [
        "neutral"
    ],
    "reason": {
        "title": "Philosophical Inquiry",
        "content": "The meaning of life is a deeply philosophical question that has been debated for centuries. It often depends on individual beliefs, cultural perspectives, and personal experiences. Some may find meaning through religion, relationships, or personal achievements, while others may see it as an existential question without a definitive answer.",
        "confidence_score": 0.9
    }
}


In [5]:
branch.to_df()

,timestamp,lion_class,role,content,ln_id,sender,recipient,metadata
0,2024-10-19T13:19:11.217771,Instruction,user,{'guidance': 'Reply concisely and informativel...,lnb50ff30b7-0e158c4-07-f6f6026812-0af42775,user,ln9b8f53c5-5-b56d8d1d7-8bb8-c13324dd02a279,"{'last_updated': {'sender': 1729358352.83078, ..."
1,2024-10-19T13:19:12.829523,AssistantResponse,assistant,{'assistant_response': {'selected': ['neutral'...,ln983f1d332ace26-974dc9-9b0-6378-236c6b993,ln9b8f53c5-5-b56d8d1d7-8bb8-c13324dd02a279,user,{'model_response': {'id': 'chatcmpl-AK7OZ0WhM1...


In [6]:
async def mock_search(query: str):
    """
    A mock search function that returns a string
    """
    return f"Search results for: {query}"


res = await branch.operate(
    instruction="Use search tool to gather information on the topic,",
    guidance="Provide a summary of the information",
    context=["What is the meaning of life?"],
    reason=True,
    actions=True,
    tools=[mock_search],
    clear_messages=True,
)

In [7]:
print(as_readable_json(res))

{
    "selected": [],
    "reason": {
        "title": "Meaning of Life Inquiry",
        "content": "The task requires gathering information on the philosophical question 'What is the meaning of life?' using a search tool. This question has been explored by various philosophical, religious, and scientific perspectives, each offering different interpretations. To provide a comprehensive summary, a search action is necessary to gather diverse viewpoints and insights.",
        "confidence_score": 0.9
    },
    "action_responses": [
        {
            "function": "mock_search",
            "arguments": {
                "query": "What is the meaning of life?"
            },
            "output": "Search results for: What is the meaning of life?"
        }
    ],
    "action_requests": [
        {
            "function": "mock_search",
            "arguments": {
                "query": "What is the meaning of life?"
            }
        }
    ],
    "action_required": true
}


In [8]:
branch.to_df()

,timestamp,lion_class,role,content,ln_id,sender,recipient,metadata
0,2024-10-19T13:19:12.891878,Instruction,user,{'guidance': 'Provide a summary of the informa...,ln25627b4-9b980d760b-4-253bcadf-ae88fc255b,user,ln9b8f53c5-5-b56d8d1d7-8bb8-c13324dd02a279,"{'last_updated': {'sender': 1729358354.793097,..."
1,2024-10-19T13:19:14.789991,AssistantResponse,assistant,"{'assistant_response': {'selected': [], 'reaso...",ln16919437-7896fa-14-0b134-e67ca0a6100465c,ln9b8f53c5-5-b56d8d1d7-8bb8-c13324dd02a279,user,{'model_response': {'id': 'chatcmpl-AK7ObW2UHD...
2,2024-10-19T13:19:14.793255,ActionRequest,assistant,"{'action_request': {'function': 'mock_search',...",ln459346f559a-a18f-5dfe-c1819cb3de-03f2f0e,ln9b8f53c5-5-b56d8d1d7-8bb8-c13324dd02a279,lnc73e76a6a514f0628-7-7-979-4099fed5e46b5e,"{'last_updated': {'sender': 1729358354.793464,..."
3,2024-10-19T13:19:14.793354,ActionResponse,assistant,{'action_request_id': 'ln459346f559a-a18f-5dfe...,ln5244f3429eac0a-1b-42-93424c-dbcedf104480,lnc73e76a6a514f0628-7-7-979-4099fed5e46b5e,ln9b8f53c5-5-b56d8d1d7-8bb8-c13324dd02a279,{'last_updated': {'content': 1729358354.793536...


In [9]:
res = await branch.chat(
    "provide 5 function calling query",
    tools=[mock_search],
    clear_messages=True,
    invoke_action=True,
    imodel=gpt4o,
)

In [10]:
print(res)

Here are five function calling queries using the specified format:

1. 
```json
{'function': 'mock_search', 'arguments': {'query': 'latest technology trends'}}
```

2. 
```json
{'function': 'mock_search', 'arguments': {'query': 'best practices for remote work'}}
```

3. 
```json
{'function': 'mock_search', 'arguments': {'query': 'top programming languages in 2023'}}
```

4. 
```json
{'function': 'mock_search', 'arguments': {'query': 'how to improve productivity'}}
```

5. 
```json
{'function': 'mock_search', 'arguments': {'query': 'benefits of meditation'}}
```


In [11]:
branch.to_df()

,timestamp,lion_class,role,content,ln_id,sender,recipient,metadata
0,2024-10-19T13:19:14.814894,Instruction,user,{'instruction': 'provide 5 function calling qu...,ln5753482c4327c121-b0f-6b-d7c32cd-cb2e94cd,user,ln9b8f53c5-5-b56d8d1d7-8bb8-c13324dd02a279,"{'last_updated': {'sender': 1729358356.226136,..."
1,2024-10-19T13:19:16.225807,AssistantResponse,assistant,{'assistant_response': 'Here are five function...,ln3fc960007-d83168494-82d0-9713b2-2bc0b2bc,ln9b8f53c5-5-b56d8d1d7-8bb8-c13324dd02a279,user,{'model_response': {'id': 'chatcmpl-AK7OcL1ck2...
2,2024-10-19T13:19:16.228436,ActionRequest,assistant,"{'action_request': {'function': 'mock_search',...",ln06d236405108708e-3-68-8caa2f652-2607231d,ln9b8f53c5-5-b56d8d1d7-8bb8-c13324dd02a279,lnc73e76a6a514f0628-7-7-979-4099fed5e46b5e,"{'last_updated': {'sender': 1729358356.228677,..."
3,2024-10-19T13:19:16.228553,ActionResponse,assistant,{'action_request_id': 'ln06d236405108708e-3-68...,lnfa97161e6-d7-c9bb5f8-c98893-4f81aa635210,lnc73e76a6a514f0628-7-7-979-4099fed5e46b5e,ln9b8f53c5-5-b56d8d1d7-8bb8-c13324dd02a279,{'last_updated': {'content': 1729358356.228758...
4,2024-10-19T13:19:16.228843,ActionRequest,assistant,"{'action_request': {'function': 'mock_search',...",ln7adb332361-279f5f-cbae7273-4db65-c888cd0,ln9b8f53c5-5-b56d8d1d7-8bb8-c13324dd02a279,lnc73e76a6a514f0628-7-7-979-4099fed5e46b5e,"{'last_updated': {'sender': 1729358356.229011,..."
5,2024-10-19T13:19:16.228919,ActionResponse,assistant,{'action_request_id': 'ln7adb332361-279f5f-cba...,ln9ce169e8e4-f5e-a014fcd1a799-616-bd64717b,lnc73e76a6a514f0628-7-7-979-4099fed5e46b5e,ln9b8f53c5-5-b56d8d1d7-8bb8-c13324dd02a279,{'last_updated': {'content': 1729358356.229078...
6,2024-10-19T13:19:16.229148,ActionRequest,assistant,"{'action_request': {'function': 'mock_search',...",lnf4c81b81e55dc4-a90b85c83f-9-8f77-0e19001,ln9b8f53c5-5-b56d8d1d7-8bb8-c13324dd02a279,lnc73e76a6a514f0628-7-7-979-4099fed5e46b5e,"{'last_updated': {'sender': 1729358356.229303,..."
7,2024-10-19T13:19:16.229214,ActionResponse,assistant,{'action_request_id': 'lnf4c81b81e55dc4-a90b85...,lnee1b54ee35cd2c-8f497-7afe-5-609b0c0f1a8f,lnc73e76a6a514f0628-7-7-979-4099fed5e46b5e,ln9b8f53c5-5-b56d8d1d7-8bb8-c13324dd02a279,{'last_updated': {'content': 1729358356.229363...
8,2024-10-19T13:19:16.229430,ActionRequest,assistant,"{'action_request': {'function': 'mock_search',...",ln728cd9-70071f5fb69-afc-1168197e-fd8afb31,ln9b8f53c5-5-b56d8d1d7-8bb8-c13324dd02a279,lnc73e76a6a514f0628-7-7-979-4099fed5e46b5e,"{'last_updated': {'sender': 1729358356.229589,..."
9,2024-10-19T13:19:16.229497,ActionResponse,assistant,{'action_request_id': 'ln728cd9-70071f5fb69-af...,ln609970-0acf8a89fe069-0c-0-2e6ffd6546d1f2,lnc73e76a6a514f0628-7-7-979-4099fed5e46b5e,ln9b8f53c5-5-b56d8d1d7-8bb8-c13324dd02a279,{'last_updated': {'content': 1729358356.229653...
